In [1]:
!pip install watermark
%load_ext watermark
# выводим информацию о версиях python и пакетов

%watermark -a "KirillSel@github.com" -d -v -p numpy,pandas,matplotlib,sklearn 

  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.8.2
    Uninstalling importlib-metadata-4.8.2:
      Successfully uninstalled importlib-metadata-4.8.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.3.6 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 2.1.2 which is incompatible.
Author: KirillSel@github.com

Python implementation: CPython
Python version       : 3.7.12
IPython version      : 5.5.0

numpy     : 1.19.5
pandas    : 1.1.5
matplotlib: 3.2.2
sklearn   : 0.0



#Загружаем пакеты

In [2]:
# загрузка пакетов: инструменты --------------------------------------------
#  работа с массивами
import numpy as np
#  фреймы данных
import pandas as pd
#  графики
import matplotlib as mpl
#  стили и шаблоны графиков на основе matplotlib
import seaborn as sns 
# перекодировка символьных показателей 
from sklearn.preprocessing import LabelEncoder
#  для таймера
import time
# загрузка пакетов: модели -------------------------------------------------
#  SVM
from sklearn.svm import SVC
# логистическая рагрессия
from sklearn.linear_model import LogisticRegression
# стандартизация
from sklearn.preprocessing import StandardScaler
# метод главных компонент
from sklearn.decomposition import PCA
# конвейеры
from sklearn.pipeline import make_pipeline
# перекрёстная проверка и метод проверочной выборки
from sklearn.model_selection import cross_val_score, train_test_split
# для перекрёстной проверки и сеточного поиска
from sklearn.model_selection import KFold,GridSearchCV
#  сводка по точности классификации
from sklearn.metrics import classification_report

In [3]:
# константы
#  ядро для генератора случайных чисел
my_seed=20
#  создаём псевдоним для короткого обращения к графикам
plt=mpl.pyplot
# настройка стиля и отображения графиков
#  примеры стилей и шаблонов графиков: 
#  http://tonysyu.github.io/raw_content/matplotlib-style-gallery/gallery.html
mpl.style.use('seaborn-whitegrid')
sns.set_palette("Set2")
# раскомментируйте следующую строку, чтобы посмотреть палитру
sns.color_palette("Set2")

[(0.4, 0.7607843137254902, 0.6470588235294118),
 (0.9882352941176471, 0.5529411764705883, 0.3843137254901961),
 (0.5529411764705883, 0.6274509803921569, 0.796078431372549),
 (0.9058823529411765, 0.5411764705882353, 0.7647058823529411),
 (0.6509803921568628, 0.8470588235294118, 0.32941176470588235),
 (1.0, 0.8509803921568627, 0.1843137254901961),
 (0.8980392156862745, 0.7686274509803922, 0.5803921568627451),
 (0.7019607843137254, 0.7019607843137254, 0.7019607843137254)]

# Загружаем данные


1. Visit [Drive folder](https://drive.google.com/drive/folders/1E9_u-v8chGmPwgROBEorD9hT609ysv_W?usp=sharing).

2. Right click on folder name -> "Add a shortcut to Drive" -> select "My Drive" -> "Add shortcut".


In [4]:
# Detect if we are in Google Colaboratory
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

from pathlib import Path
# Determine the locations of auxiliary libraries and datasets.
# AUX_DATA_ROOT is where 'notmnist.py', 'animation.py' and 'tiny-imagenet-2020.zip' are.
if IN_COLAB:
    google.colab.drive.mount("/content/drive")
    
    # Change this if you created the shortcut in a different location
    AUX_DATA_ROOT = Path("/content/drive/My Drive/data")
    
    assert AUX_DATA_ROOT.is_dir(), "Have you forgot to 'Add a shortcut to Drive'?"
    
    import sys
    sys.path.insert(0, str(AUX_DATA_ROOT))
else:
    AUX_DATA_ROOT = Path(".")

Mounted at /content/drive


In [5]:
import os 
DF_raw = pd.read_csv(os.path.join(AUX_DATA_ROOT, 'default_of_credit_card_clients.csv'))

print('Число строк и столбцов в наборе данных:\n', DF_raw.shape)

Число строк и столбцов в наборе данных:
 (30000, 24)


In [6]:
DF_raw.dtypes

LIMIT_BAL    int64
SEX          int64
EDUCATION    int64
MARRIAGE     int64
AGE          int64
PAY_0        int64
PAY_2        int64
PAY_3        int64
PAY_4        int64
PAY_5        int64
PAY_6        int64
BILL_AMT1    int64
BILL_AMT2    int64
BILL_AMT3    int64
BILL_AMT4    int64
BILL_AMT5    int64
BILL_AMT6    int64
PAY_AMT1     int64
PAY_AMT2     int64
PAY_AMT3     int64
PAY_AMT4     int64
PAY_AMT5     int64
PAY_AMT6     int64
Y            int64
dtype: object

Отложим 15% наблюдений для прогноза. 

In [7]:
# наблюдения для моделирования
DF = DF_raw.sample(frac=0.85, random_state=my_seed)
# отложенные наблюдения
DF_predict = DF_raw.drop(DF.index)

DF - TRAIN
DF_predict - TEST

Все объясняющие переменные набора данных непрерывные, кроме SEX, EDUCATION и MARRIAGE. Рассчитаем для них описательные статистики. 

In [8]:
DF.drop(['SEX', 'EDUCATION', 'MARRIAGE'], axis=1).describe()

,LIMIT_BAL,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,Y
count,25500.000000,25500.000000,25500.000000,25500.000000,25500.000000,25500.000000,25500.000000,25500.000000,25500.000000,25500.000000,2.550000e+04,25500.000000,25500.000000,25500.000000,25500.000000,2.550000e+04,25500.000000,25500.000000,25500.000000,25500.000000,25500.000000
mean,168077.242353,35.486471,-0.015608,-0.131882,-0.166000,-0.218196,-0.261647,-0.290118,51399.370471,49355.392471,4.720979e+04,43439.331490,40425.389725,39032.859098,5699.709020,5.963576e+03,5221.213333,4820.416824,4850.173843,5255.832627,0.220588
std,129837.678356,9.213824,1.126387,1.201093,1.199212,1.169805,1.135101,1.149237,73891.950872,71405.114321,6.946620e+04,64311.588682,60587.830600,59484.804698,16687.930104,2.327991e+04,16875.401546,15353.273877,15584.945606,17773.623966,0.414651
min,10000.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-165580.000000,-69777.000000,-6.150600e+04,-81334.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,50000.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,3538.500000,2983.000000,2.733750e+03,2390.750000,1780.000000,1272.750000,1000.000000,8.500000e+02,390.000000,298.000000,265.000000,133.000000,0.000000
50%,140000.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22407.000000,21241.500000,2.017750e+04,19158.500000,18275.000000,17199.000000,2103.000000,2.010000e+03,1800.000000,1500.000000,1500.000000,1500.000000,0.000000
75%,240000.000000,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,67255.750000,64150.000000,6.049100e+04,54715.250000,50320.000000,49355.500000,5019.000000,5.000000e+03,4523.250000,4030.750000,4078.000000,4045.250000,0.000000
max,800000.000000,79.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,653062.000000,743970.000000,1.664089e+06,706864.000000,823540.000000,699944.000000,873552.000000,1.684259e+06,889043.000000,621000.000000,426529.000000,528666.000000,1.000000


Для категориальных переменных SEX, EDUCATION и MARRIAGE подсчитаем частоты значений


In [9]:
count_sex = DF['SEX'].value_counts()
pd.DataFrame(count_sex)

,SEX
2,15408
1,10092


In [10]:
count_edu = DF['EDUCATION'].value_counts()
pd.DataFrame(count_edu)

,EDUCATION
2,11894
1,9055
3,4141
5,253
4,100
6,46
0,11


In [11]:
count_mar = DF['MARRIAGE'].value_counts()
pd.DataFrame(count_mar)

,MARRIAGE
2,13585
1,11605
3,261
0,49


Теперь перекодируем категориальные переменные SEX, EDUCATION и MARRIAGE с помощью OneHotEncoder

In [12]:
from sklearn.preprocessing import OneHotEncoder

nom_col_names=['SEX', 'EDUCATION', 'MARRIAGE']

one_hot=OneHotEncoder()

recoded=one_hot.fit_transform(DF[nom_col_names]).toarray()
clmns=one_hot.get_feature_names(nom_col_names)
df_dummy_nom=pd.DataFrame(recoded, columns=clmns)

print(df_dummy_nom.shape)
df_dummy_nom.head()

(25500, 13)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,SEX_1,SEX_2,EDUCATION_0,EDUCATION_1,EDUCATION_2,EDUCATION_3,EDUCATION_4,EDUCATION_5,EDUCATION_6,MARRIAGE_0,MARRIAGE_1,MARRIAGE_2,MARRIAGE_3
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Сконкатенируем наши датафреймы и удалим ненужные столбцы

In [13]:
DF_num = pd.concat([DF.loc[:,DF.dtypes=='int64'].reset_index(),df_dummy_nom], axis=1)
DF_num = DF_num.drop(['SEX', 'EDUCATION', 'MARRIAGE', 'index'], axis=1)
print('Размерность обучающего фрейма после перекодировки:', DF_num.shape)

print()
DF_num.head()

Размерность обучающего фрейма после перекодировки: (25500, 34)



,LIMIT_BAL,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,Y,SEX_1,SEX_2,EDUCATION_0,EDUCATION_1,EDUCATION_2,EDUCATION_3,EDUCATION_4,EDUCATION_5,EDUCATION_6,MARRIAGE_0,MARRIAGE_1,MARRIAGE_2,MARRIAGE_3
0,60000,28,1,2,0,0,0,0,47262,46226,47557,48464,49532,50552,0,2100,2000,2000,2000,3000,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,30000,34,2,-1,2,2,2,2,99568,32326,31840,37075,37662,36904,33000,33000,5810,1325,0,3095,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,50000,30,0,0,2,2,2,0,45948,50813,45393,49351,45473,48595,6012,10,8212,1200,4008,4407,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,240000,42,2,2,2,2,2,2,259511,258208,245397,245014,209834,195913,11000,0,12300,14288,0,6887,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,300000,28,0,0,2,2,2,2,275855,286812,280337,318248,316147,303666,15000,7000,50000,12000,0,13000,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Теперь сделаем то же самое на отложенных данных DF_predict

In [14]:
nom_col_names=['SEX', 'EDUCATION', 'MARRIAGE']

recoded = one_hot.fit_transform(DF_predict[nom_col_names]).toarray()
clmns = one_hot.get_feature_names(nom_col_names)
df_dummy_nom = pd.DataFrame(recoded, columns=clmns)

print(df_dummy_nom.shape)
df_dummy_nom.head()

(4500, 13)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,SEX_1,SEX_2,EDUCATION_0,EDUCATION_1,EDUCATION_2,EDUCATION_3,EDUCATION_4,EDUCATION_5,EDUCATION_6,MARRIAGE_0,MARRIAGE_1,MARRIAGE_2,MARRIAGE_3
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


Соединим и удалим ненужное

In [15]:
DF_predict = pd.concat([DF_predict.loc[:, DF_predict.dtypes=='int64'].reset_index(), df_dummy_nom], axis=1)
DF_predict = DF_predict.drop(['SEX', 'EDUCATION', 'MARRIAGE', 'index'], axis=1)
print('Размерность тестового фрейма после перекодировки:', DF_predict.shape)

print()
DF_predict.head()

Размерность тестового фрейма после перекодировки: (4500, 34)



,LIMIT_BAL,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,Y,SEX_1,SEX_2,EDUCATION_0,EDUCATION_1,EDUCATION_2,EDUCATION_3,EDUCATION_4,EDUCATION_5,EDUCATION_6,MARRIAGE_0,MARRIAGE_1,MARRIAGE_2,MARRIAGE_3
0,90000,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,50000,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,140000,28,0,0,2,0,0,0,11285,14096,12108,12211,11793,3719,3329,0,432,1000,1000,1000,0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,50000,23,1,2,0,0,0,0,50614,29173,28116,28771,29531,30211,0,1500,1100,1200,1300,1100,0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,320000,49,0,0,0,-1,-1,-1,253286,246536,194663,70074,5856,195599,10358,10000,75940,20000,195599,50000,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


#Стандартизация и переход к главным компонентам

In [61]:
DF = DF_num

In [63]:
# стандартизация
sc=StandardScaler()
X_train_std=sc.fit_transform(DF.iloc[:,2:].values)
# оцениваем объяснённую главными компонентами дисперсию
pca=PCA()
X_train_pca=pca.fit_transform(X_train_std)
# считаем доли объяснённой дисперсии
frac_var_expl=pca.explained_variance_ratio_
print('Доли объяснённой дисперсии по компонентам в PLS:\n',
      np.around(frac_var_expl,3),
      '\nОбщая сумма долей:',np.around(sum(frac_var_expl),3))

Доли объяснённой дисперсии по компонентам в PLS:
 [0.204 0.122 0.068 0.061 0.05  0.048 0.038 0.032 0.032 0.031 0.031 0.031
 0.03  0.029 0.028 0.027 0.027 0.023 0.022 0.02  0.012 0.008 0.008 0.006
 0.004 0.002 0.001 0.001 0.001 0.    0.    0.   ] 
Общая сумма долей: 1.0


Таким образом, первые 6 главных компонент объясняют 55.3% разброса 22 объясняющих переменных.Теперь объединим функции-преобразователи и оценщики в конвейер с помощью Pipeline и оценим точность логистической регрессии с помощью перекрёстной проверки.

In [64]:
# данные для обучения моделей
X_train=DF.drop(['Y'], axis=1)
y_train=DF['Y']

1 пайплайн с Логистической регрессией

In [65]:
pipe_lr = make_pipeline(StandardScaler(), LogisticRegression(random_state=my_seed))

param_grid = {"logisticregression__C" : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000], "logisticregression__penalty" : ["l1", "l2"]}
kfold = KFold(n_splits = 5, random_state = my_seed, shuffle = True)

gs = GridSearchCV(estimator=pipe_lr, param_grid = param_grid, scoring = 'accuracy', refit=True, cv = kfold, n_jobs=-1)

tic = time.perf_counter()
gs = gs.fit(X_train,y_train)

toc = time.perf_counter()
print(f"Сеточный поиск занял {toc-tic:0.2f} секунд",sep='')

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
45 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
45 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  Fil

Сеточный поиск занял 8.72 секунд


Варнинги говорят о том, что некоторые параметры не могут быть использованы вместе и GridSearch автоматически их пропускает, присваивая им score = NAN

In [66]:
print(f'Best Score for pipeline with Logistic Regression: {np.around(gs.best_score_,3)}')

Best Score for pipeline with Logistic Regression: 0.811


In [67]:
print("Penalty =", gs.best_estimator_.get_params()['logisticregression__penalty'])
print("C = ", gs.best_estimator_.get_params()['logisticregression__C'])


print("Params", gs.cv_results_['params'])

print("Mean Test Scores", gs.cv_results_['mean_test_score'])

Penalty = l2
C =  10
Params [{'logisticregression__C': 0.0001, 'logisticregression__penalty': 'l1'}, {'logisticregression__C': 0.0001, 'logisticregression__penalty': 'l2'}, {'logisticregression__C': 0.001, 'logisticregression__penalty': 'l1'}, {'logisticregression__C': 0.001, 'logisticregression__penalty': 'l2'}, {'logisticregression__C': 0.01, 'logisticregression__penalty': 'l1'}, {'logisticregression__C': 0.01, 'logisticregression__penalty': 'l2'}, {'logisticregression__C': 0.1, 'logisticregression__penalty': 'l1'}, {'logisticregression__C': 0.1, 'logisticregression__penalty': 'l2'}, {'logisticregression__C': 1, 'logisticregression__penalty': 'l1'}, {'logisticregression__C': 1, 'logisticregression__penalty': 'l2'}, {'logisticregression__C': 10, 'logisticregression__penalty': 'l1'}, {'logisticregression__C': 10, 'logisticregression__penalty': 'l2'}, {'logisticregression__C': 100, 'logisticregression__penalty': 'l1'}, {'logisticregression__C': 100, 'logisticregression__penalty': 'l2'},

2 пайплайн с Random Forest

In [79]:
from sklearn.ensemble import RandomForestClassifier

pipe_rf = make_pipeline(StandardScaler(), RandomForestClassifier(random_state=my_seed, max_features='auto'))

param_grid = { 
    'randomforestclassifier__n_estimators': [200, 500],
    'randomforestclassifier__max_depth' : [5, 7, 9],
    'randomforestclassifier__criterion' :['gini', 'entropy']
}

kfold = KFold(n_splits = 5, random_state = my_seed, shuffle = True)

gs = GridSearchCV(estimator=pipe_rf, param_grid = param_grid, scoring = 'accuracy', refit = True, cv = kfold, n_jobs = -1)

tic = time.perf_counter()
gs = gs.fit(X_train,y_train)

toc = time.perf_counter()
print(f"Сеточный поиск занял {toc-tic:0.2f} секунд",sep='')

Сеточный поиск занял 479.31 секунд


In [80]:
print(f'Best Score for pipeline with Random Forest:: {np.around(gs.best_score_,3)}')

Best Score for pipeline with Random Forest:: 0.821


In [81]:
print(gs.best_estimator_.get_params()['randomforestclassifier__n_estimators'])
print(gs.best_estimator_.get_params()['randomforestclassifier__max_features'])
print(gs.best_estimator_.get_params()['randomforestclassifier__max_depth'])
print(gs.best_estimator_.get_params()['randomforestclassifier__criterion'])


print(gs.cv_results_['params'])

print(gs.cv_results_['mean_test_score'])

200
auto
9
gini
[{'randomforestclassifier__criterion': 'gini', 'randomforestclassifier__max_depth': 5, 'randomforestclassifier__n_estimators': 200}, {'randomforestclassifier__criterion': 'gini', 'randomforestclassifier__max_depth': 5, 'randomforestclassifier__n_estimators': 500}, {'randomforestclassifier__criterion': 'gini', 'randomforestclassifier__max_depth': 7, 'randomforestclassifier__n_estimators': 200}, {'randomforestclassifier__criterion': 'gini', 'randomforestclassifier__max_depth': 7, 'randomforestclassifier__n_estimators': 500}, {'randomforestclassifier__criterion': 'gini', 'randomforestclassifier__max_depth': 9, 'randomforestclassifier__n_estimators': 200}, {'randomforestclassifier__criterion': 'gini', 'randomforestclassifier__max_depth': 9, 'randomforestclassifier__n_estimators': 500}, {'randomforestclassifier__criterion': 'entropy', 'randomforestclassifier__max_depth': 5, 'randomforestclassifier__n_estimators': 200}, {'randomforestclassifier__criterion': 'entropy', 'random

3 пайплайн с KNN

Построим несколько вариантов модели KNN с различным количество соседей

In [74]:
from sklearn.neighbors import KNeighborsClassifier

pipe_knn = make_pipeline(StandardScaler(), PCA(n_components = 6), KNeighborsClassifier())

neighbours = [3, 4, 5, 6, 7, 8 ,11, 12, 13, 14, 15, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]
param_grid=[{'kneighborsclassifier__n_neighbors' : neighbours}]


kfold=KFold(n_splits=5,random_state=my_seed,shuffle=True)

gs=GridSearchCV(estimator=pipe_knn, param_grid = param_grid, scoring = 'accuracy', refit=True, cv=kfold, n_jobs=-1)

tic = time.perf_counter()
# запускаем сеточный поиск
gs=gs.fit(X_train,y_train)
# таймер
toc=time.perf_counter()
print(f"Сеточный поиск занял {toc-tic:0.2f} секунд",sep='')

Сеточный поиск занял 38.65 секунд


In [75]:
print(f'Best Score for pipeline with KNN: {np.around(gs.best_score_,3)}')

Best Score for pipeline with KNN: 0.804


In [76]:
gs.best_estimator_.get_params()['kneighborsclassifier__n_neighbors']

32

In [77]:
gs.cv_results_['params']

[{'kneighborsclassifier__n_neighbors': 3},
 {'kneighborsclassifier__n_neighbors': 4},
 {'kneighborsclassifier__n_neighbors': 5},
 {'kneighborsclassifier__n_neighbors': 6},
 {'kneighborsclassifier__n_neighbors': 7},
 {'kneighborsclassifier__n_neighbors': 8},
 {'kneighborsclassifier__n_neighbors': 11},
 {'kneighborsclassifier__n_neighbors': 12},
 {'kneighborsclassifier__n_neighbors': 13},
 {'kneighborsclassifier__n_neighbors': 14},
 {'kneighborsclassifier__n_neighbors': 15},
 {'kneighborsclassifier__n_neighbors': 25},
 {'kneighborsclassifier__n_neighbors': 26},
 {'kneighborsclassifier__n_neighbors': 27},
 {'kneighborsclassifier__n_neighbors': 28},
 {'kneighborsclassifier__n_neighbors': 29},
 {'kneighborsclassifier__n_neighbors': 30},
 {'kneighborsclassifier__n_neighbors': 31},
 {'kneighborsclassifier__n_neighbors': 32},
 {'kneighborsclassifier__n_neighbors': 33},
 {'kneighborsclassifier__n_neighbors': 34},
 {'kneighborsclassifier__n_neighbors': 35},
 {'kneighborsclassifier__n_neighbors':

In [78]:
gs.cv_results_['mean_test_score']

array([0.77078431, 0.78913725, 0.78305882, 0.79321569, 0.79180392,
       0.79498039, 0.79776471, 0.79807843, 0.79827451, 0.79960784,
       0.79976471, 0.80247059, 0.80356863, 0.80317647, 0.80301961,
       0.80337255, 0.80360784, 0.80352941, 0.80431373, 0.80341176,
       0.80321569, 0.8032549 , 0.80376471, 0.80376471, 0.80337255])

PREDICT

Лучшй пайплайн моделей оказался Random Forest и показало 82% accuracy на train

In [56]:
X_test = DF_predict.drop(['Y'], axis = 1)
y_test = DF_predict['Y']

In [83]:
from sklearn.metrics import accuracy_score

y_pred = gs.best_estimator_.predict(X_test)

print(f'ACCURACY FOR THE BEST MODEL IS : {accuracy_score(y_test, y_pred)}')
print(classification_report(DF_predict['Y'], y_pred))

ACCURACY FOR THE BEST MODEL IS : 0.818
              precision    recall  f1-score   support

           0       0.83      0.96      0.89      3489
           1       0.70      0.34      0.45      1011

    accuracy                           0.82      4500
   macro avg       0.76      0.65      0.67      4500
weighted avg       0.80      0.82      0.79      4500



К сожалению точности в 96% не получилось достичь, самая лучшая точность это 82%, которую мне получиось достичь на тесте